# Recommender System

In [1]:
import pandas as pd
import numpy as np
import spacy

---

In [5]:
jobs = pd.read_csv('../data/job_postings.csv')

In [7]:
jobs.head(1)

,Unnamed: 0,date_added,job_description,job_title,job_type,location,organization,skills,has_email,has_phone_number,skills_len
0,0,11/11/2016,"If you are an EDI Analyst with experience, ple...",Analyst,Full Time,Northeast United States,CyberCoders,edi trustedlink as2 van,0.0,0.0,9.0


In [20]:
jobs = jobs.drop(columns=['Unnamed: 0', 'date_added', 'organization', 'has_email', 'has_phone_number', 'skills_len', 'job_type'])

In [2]:
nlp = spacy.load("en_core_web_sm")

In [21]:
# We've lost the proper alignmnet at some point during the data cleaning
jobs['job_description'].isna().sum()

25

In [22]:
jobs[jobs['job_description'].isna()]

,job_description,job_title,location,skills
902,NaN,Engineer,Northeast United States,c++ access business intelligence data structur...
4220,NaN,Developer,Northeast United States,.net access amazon android aws business intell...
4299,NaN,NaN,NaN,NaN
6758,NaN,NaN,NaN,NaN
9551,NaN,NaN,NaN,NaN
9991,NaN,NaN,NaN,NaN
10453,NaN,NaN,NaN,NaN
10785,NaN,NaN,NaN,NaN
10789,NaN,NaN,NaN,NaN
10951,NaN,NaN,NaN,NaN


In [23]:
jobs.isna().sum()

job_description     25
job_title           19
location            19
skills             390
dtype: int64

In [24]:
jobs.dropna(inplace=True)
jobs.reset_index(drop=True, inplace=True)

In [30]:
list_docs = []
for i in range(len(jobs)):
    if jobs['job_description'][i]:
        doc = nlp("u'" + jobs['job_description'][i] + "'")
        list_docs.append(doc)

In [31]:
def calculate_sim_with_spacy(nlp, df, user_text, n=6):
    list_sim = []
    doc1 = nlp("u'" + user_text + "'")
    for i in df.index:
        try:
            doc2 = list_docs[i]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, score))
        except:
            continue
    return list_sim

In [36]:
list_docs[0]

u'If you are an EDI Analyst with experience, please read on!We are a strong, long standing company looking for an EDI Analyst for our team. You must have 3+ years of EDI experience in a TrustedLink for i environment. Your role will work with our finance department identifying trading partners, work closely with external customers and be the EDI liaison across the company. You will also monitor AS2 and VAN communications, correct errors and incoming data.What You Need for this PositionRequirements:3+ years of EDI experience3+ years of TrustedLink for i Experience, iSeries/AS400Experience with VAN and AS2 communicationsWhat's In It for YouWe offer a strong compensation package and benefits!Local Candidates ONLY please.So, if you are an EDI Analyst with experience, please apply today!Applicants must be authorized to work in the U.S.Please apply directly to by clicking 'Click Here to Apply' with your Word resume!Looking forward to receiving your resume and going over the position in more d

In [40]:
sim = calculate_sim_with_spacy(nlp, jobs['job_description'], 'java developer')

<ipython-input-31-4bb41cda221f>:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  score = doc1.similarity(doc2)


In [50]:
sims = [(s[1],s[2]) for s in sim]

In [58]:
df = pd.DataFrame(sims).sort_values(by=1).reset_index(drop=True)

In [59]:
df.head(5)

,0,1
0,"(u'Please, share, resume, to, , or, Call, me,...",0.030524
1,"(u'Job, Summary, :, The, Manager, of, Informat...",0.031012
2,"(u'E*Pro, Consulting, service, offerings, incl...",0.062507
3,"(u'Role, :, Team, center, PLM, developerLocati...",0.072988
4,"(u'US, citizens, and, those, authorized, to, w...",0.104845


In [79]:
df.tail(10)

,0,1
16255,"(u'Software, Architect, 6, MonthsDearborn, ,, ...",0.545974
16256,"(u'PHP, Developer, ')",0.549071
16257,"(u'Liferay, Website, hosted, on, version, 6.2L...",0.550896
16258,"(u'5, years, experience, working, on, Middlewa...",0.573112
16259,"(u'""MusthavepriorDataconversionexperienceproje...",0.577609
16260,"(u'Network, Storage, Engineer6, monthsAustin, ...",0.579050
16261,"(u'SR, Android, Developer, ')",0.601328
16262,"(u'Java, and, Liferay, ')",0.622916
16263,"(u'systems, engineer, RF, communica, Design, t...",0.690385
16264,"(u'Java, planner, ')",0.750848


In [80]:
df.tail(10).loc[16259, 0]

u'"MusthavepriorDataconversionexperienceprojects;beabletounderstandgeneralprocesses.RequiresSFDCknowledgeforconverstionofSFDCtoanothertechn ology.Mustbefamiliarwithstandardvs.non-standarddatatablesinSFDCandcommunicatetotheprojectteamwhatcustomizationsareinplace."'